### [GLOBAL IMPORTS]

In [47]:
from fastai.data.external import untar_data, URLs
from fastai.data.core import L

from fastai.data.block import (
    DataBlock,
)

from fastai.data.transforms import RandomSplitter

from fastai.text.all import (
    defaults,
    # file handler
    get_text_files,
    # tabular util
    Tokenizer, WordTokenizer, SubwordTokenizer, Numericalize,
    LMDataLoader, 
    # data block
    TextBlock,
    # model
    AWD_LSTM,
    # metric
    Perplexity, accuracy,
    # learner
    language_model_learner,
    # debug log
    first, coll_repr
)

from fastai.text.core import replace_rep

from functools import partial

#### Inspect source code with `??`

In [3]:

# inspect source
# Tokenizer??
replace_rep??

Signature: replace_rep(t)
Source:   
def replace_rep(t):
    "Replace repetitions at the character level: cccc -- TK_REP 4 c"
    def _replace_rep(m):
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    return _re_rep.sub(_replace_rep, t)
File:      ~/miniconda3/envs/torch-gpu/lib/python3.9/site-packages/fastai/text/core.py
Type:      function


### NLP Deep Dive : RNNs

Self-supervised learning is training a model using labels that are :
- EMBEDDED in the independent variable
- rather than requiring EXTERNAL labels
- example : training the model to predict the next word in a text

ULMFit - Universal Language Model Fine-tuning
- Fine tuning the :
    - (sequence based) language model prior to
    - fine-tuning the classification model yiels BETTER results

#### Text Processing

Predicting sentence length isn't obvious ... correlated to human breathe?
- Sentences can be of different LENGTHS
- Documents can be LONG

Review our background with how a single categorical variable can be used as an
indpendent variable, here's the approach we took for a single categorical var :
- 1 - Make a list of all the possible levels of that categorical var -- vocab --
- 2 - Replace each level with it's `index` in the -- vocab --
- 3 - Create an `embedding matrix` for this containing a row for each item
      .i.e for each item in the -- vocab --
- 4 - Use this `embedding matrix` as the first layer of a neural network  

```sh
        A dedicated **embedding matrix** can take as inputs the raw -- vocab --  
        indexes created in step 2;  
            - this is equivalent to  
            - but FASTER and more EFFICIENT than a matrix that takes as input  
            one-hot-encoded vectors representing the indexes  
```

We can do the same thing ^ with TEXT!  What is new is the idea of a sequence

- 1 - [ Tokenization ]
    - convert text into a list of (depending on granularity) :  
      - characters  
      - substrings - (GPT, HuggingFace)  
      - words  
- 2 - [ Numericalization ]  
    - -- vocab -- list hashed to an index number lookup
- 3 - Language Model [ Data Loader Creation ]   
    - `LMDataLoader` handles creating  
      - `dependent` variable that is  
      - `offset` from the `independent` by ONE `token`  
    - Also handles details such as :  
      - shuffling the training data so that the independent and dependent  
        variable maintain their structure as required  
      - latent breathe?  
- 4 - [ Language Model Creation ]  
    - RNN  
      - handles INPUT lists that can be of ARBITRARY LENGTH  


### -- Tokenization --

Resolution

- 1 - Character
    - split into INDIVDUAL CHARS
- 2 - Subword
    - split into SMALLER parts
    - based on most COMMONLY occuring substrings
        - "occasion" => "o" "c" "ca" "sion"
- 3 - Word
    - apply language specific separator like 'white' space
    - generally punctuation marks are SEPARATE tokens
        - as opposed to totally NEW words

In [4]:
path = untar_data(URLs.IMDB)

files = get_text_files(path, folders=['train', 'test', 'unsup'])
txt = files[0].open().read()

files, txt[:175]

((#100000) [Path('/Users/mton/.fastai/data/imdb/test/neg/1821_4.txt'),Path('/Users/mton/.fastai/data/imdb/test/neg/9487_1.txt'),Path('/Users/mton/.fastai/data/imdb/test/neg/4604_4.txt'),Path('/Users/mton/.fastai/data/imdb/test/neg/2828_2.txt'),Path('/Users/mton/.fastai/data/imdb/test/neg/10890_1.txt'),Path('/Users/mton/.fastai/data/imdb/test/neg/3351_4.txt'),Path('/Users/mton/.fastai/data/imdb/test/neg/8070_2.txt'),Path('/Users/mton/.fastai/data/imdb/test/neg/1027_4.txt'),Path('/Users/mton/.fastai/data/imdb/test/neg/8248_3.txt'),Path('/Users/mton/.fastai/data/imdb/test/neg/4290_4.txt')...],
 "Alan Rickman & Emma Thompson give good performances with southern/New Orleans accents in this detective flick. It's worth seeing for their scenes- and Rickman's scene with Hal")

In [5]:
spacy = WordTokenizer()
toks = first(spacy([txt]))

print(coll_repr(toks, 30))

# The char '.' is terminated in a sentence, but not the '1.00' acronym
# Tokenization logic needs to handle very subtle context
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#121) ['Alan','Rickman','&','Emma','Thompson','give','good','performances','with','southern','/','New','Orleans','accents','in','this','detective','flick','.','It',"'s",'worth','seeing','for','their','scenes-','and','Rickman',"'s",'scene'...]


(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

#### -- Common -- [ Prefixes ( xx ) ]

xx - not common prefix, these are special tokens
- xxbos : Beginning of stream
    - this token indicates the model will learn it needs to "forget" what was  
    said previously and focus on upcoming words
- xxmaj : Indicates the next word begins with a capital 
  (we lower cased everything)
- xxunk : Indicates a word is unknown
- xxrep : !!!!! => `repeated char token` + `!` so we can count repeats as  
  opposed to treating them as unique
- xxwrep : for repeated words as opposed to characters

In [6]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt)), 31)

(#139) ['xxbos','xxmaj','alan','xxmaj','rickman','&','xxmaj','emma','xxmaj','thompson'...] 31


#### -- FastAi -- [ Text Processing Rules ]

In [7]:
# inspect source code
replace_rep??

# check default rules
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

Signature: replace_rep(t)
Source:   
def replace_rep(t):
    "Replace repetitions at the character level: cccc -- TK_REP 4 c"
    def _replace_rep(m):
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    return _re_rep.sub(_replace_rep, t)
File:      ~/miniconda3/envs/torch-gpu/lib/python3.9/site-packages/fastai/text/core.py
Type:      function


#### -- Subword -- [Vocabulary Size]

Subword tokenization provides an easy way to :
- easily scale between character and word tokenization
- handles EVERY human language (not just white space separated)
    - including music and genomic sequences

Vocabulary Size is a trade-off between :

- Larger - fewer tokens per sentences
    - faster training
    - less state
    - downside : LARGER EMBEDDING MATRIX
        - requires MORE data to LEARN

In [8]:
txts = L(o.open().read() for o in files[:2000])
txts

(#2000) ["Alan Rickman & Emma Thompson give good performances with southern/New Orleans accents in this detective flick. It's worth seeing for their scenes- and Rickman's scene with Hal Holbrook. These three actors mannage to entertain us no matter what the movie, it seems. The plot for the movie shows potential, but one gets the impression in watching the film that it was not pulled off as well as it could have been. The fact that it is cluttered by a rather uninteresting subplot and mostly uninteresting kidnappers really muddles things. The movie is worth a view- if for nothing more than entertaining performances by Rickman, Thompson, and Holbrook.",'I have seen this movie and I did not care for this movie anyhow. I would not think about going to Paris because I do not like this country and its national capital. I do not like to learn french anyhow because I do not understand their language. Why would I go to France when I rather go to Germany or the United Kingdom? Germany and the U

In [9]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [10]:
subword(1000)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tmp/texts.out --vocab_size=1000 --model_prefix=tmp/spm --character_coverage=0.99999 --model_type=unigram --unk_id=9 --pad_id=-1 --bos_id=-1 --eos_id=-1 --minloglevel=2 --user_defined_symbols=▁xxunk,▁xxpad,▁xxbos,▁xxeos,▁xxfld,▁xxrep,▁xxwrep,▁xxup,▁xxmaj --hard_vocab_limit=false


'▁A l an ▁R ick man ▁ & ▁E mm a ▁Th om p son ▁give ▁good ▁performance s ▁with ▁so u ther n / N e w ▁O r le an s ▁a c cent s ▁in ▁this ▁de'

In [11]:
subword(200)

'▁A l an ▁ R i ck m an ▁ & ▁ E m m a ▁ T h o m p s on ▁g i ve ▁g o o d ▁p er f or m an ce s ▁with'

In [12]:
subword(10_000)

"▁Alan ▁Rick man ▁ & ▁Emma ▁Thompson ▁give ▁good ▁performances ▁with ▁southern / N e w ▁O rleans ▁accents ▁in ▁this ▁detective ▁flick . ▁It ' s ▁worth ▁seeing ▁for ▁their ▁scenes - ▁and ▁Rick man ' s ▁scene ▁with"

##### Numericalization with Fastai

Numericalization is the process of mapping tokens to integers
- 1 - vocabulary : list of all possible levels of categorical variable
    - RGB captures visual pixel values!
- 2 - replace each level with it's index in the vocab
    - each R, G, B channel has value between 0 - 255

In [13]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#139) ['xxbos','xxmaj','alan','xxmaj','rickman','&','xxmaj','emma','xxmaj','thompson','give','good','performances','with','southern','/','xxmaj','new','xxmaj','orleans','accents','in','this','detective','flick','.','xxmaj','it',"'s",'worth','seeing'...]


In [14]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#139) ['xxbos','xxmaj','alan','xxmaj','rickman','&','xxmaj','emma','xxmaj','thompson'...]

In [15]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab, 20)


"(#1984) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','and','a','to','of','i','it','is','in'...]"

In [16]:
nums = num(toks)[:20]
nums

TensorText([   2,    8,    0,    8, 1442,  234,    8,    0,    8,    0,  199,
              64,  731,   29,    0,  122,    8,  253,    8,    0])

In [17]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj xxunk xxmaj rickman & xxmaj xxunk xxmaj xxunk give good performances with xxunk / xxmaj new xxmaj xxunk'

##### Putting Our Texts into Batches for a Language Model

With IMAGEs for batching we needed to :

- RESIZE height and width
- so we could group and stack them in a single tensor

With TEXT
- can't resize arbitrarily varied length to fix length
- char order matters to predict next token
- each NEW batch MUST begin precisely where the old batch finished



In [22]:
nums200 = toks200.map(num)

dl = LMDataLoader(nums200)

x,y = first(dl)
x.shape, y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [25]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj xxunk xxmaj rickman & xxmaj xxunk xxmaj xxunk give good performances with xxunk / xxmaj new xxmaj xxunk'

In [26]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj xxunk xxmaj rickman & xxmaj xxunk xxmaj xxunk give good performances with xxunk / xxmaj new xxmaj xxunk accents'

## -- Training a Text Classifier --

- 1 - fine-tune our `language model` trained on Wikipedia
- 2 - use that model to train our `classifier`

### Language Model Using DataBlock

In [42]:
get_imdb = partial(
    get_text_files,
    folders = ['train', 'test', 'unsup']
)

In [44]:
# FileNotFoundError: [Errno 2] No such file or directory: 
# '.fastai/data/imdb_tok/counter.pkl'
# This can occur if we cancel this tok process : 
# - it'll cache in a malformed
# - when we rerun, it only checks that the 'imdb_tok' folder exists
# - and tries to load the `counter.pkl` that never actually got completed
dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, 
    splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [45]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos i loved xxmaj darkwing xxmaj duck when i was 5 years old and i still love it now ! xxmaj it 's interesting how most cartoons from childhood seem lame when i watch them now but xxmaj darkwing xxmaj duck remains funny and entertaining . xxmaj the witty dialog , action , diverse characters , and style made this show unique from most xxmaj disney cartoons . xxmaj if they ever remade the series i would most definitely watch","i loved xxmaj darkwing xxmaj duck when i was 5 years old and i still love it now ! xxmaj it 's interesting how most cartoons from childhood seem lame when i watch them now but xxmaj darkwing xxmaj duck remains funny and entertaining . xxmaj the witty dialog , action , diverse characters , and style made this show unique from most xxmaj disney cartoons . xxmaj if they ever remade the series i would most definitely watch it"
1,"sincerity . xxmaj and i got ta admit that even though xxmaj i 'm quite a huge fan of xxmaj uwe xxmaj boll 's work , i personally was n't 100 % sure either what to expect from a project like this … \n\n xxmaj but now that i finally got my hands on a xxup dvd of said film and ultimately got to see xxup tunnel xxup rats ( i missed the flick 's short theatrical run here in",". xxmaj and i got ta admit that even though xxmaj i 'm quite a huge fan of xxmaj uwe xxmaj boll 's work , i personally was n't 100 % sure either what to expect from a project like this … \n\n xxmaj but now that i finally got my hands on a xxup dvd of said film and ultimately got to see xxup tunnel xxup rats ( i missed the flick 's short theatrical run here in xxmaj"


### Fine-Tuning the Language Model



In [48]:
learn = language_model_learner(
    dls_lm,
    AWD_LSTM,
    drop_mult=0.3,
    metrics=[accuracy, Perplexity()]
).to_fp16()